In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.6 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import glob
import numpy as np
import cv2
from tqdm import tqdm
from ultralytics import YOLO
from scipy.optimize import differential_evolution
import time
import yaml
import torch
import random

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MN-20-Credit/Aerial-YOLO-DOTA/src

/content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/Aerial-YOLO-DOTA/src


# One Pixel Attack

In [ ]:
# =========  PATHS AND CONFIGURATION =========
MODEL_PATH      = "/content/drive/MyDrive/MN-20-Credit/models/yolov9/best.pt"
VAL_IMAGES_DIR  = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val"
VAL_LABELS_DIR  = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/labels/val"

ATTACK_ROOT     = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/de_one_pixel_attack"
ATTACK_IMG_DIR  = os.path.join(ATTACK_ROOT, "images/val")
ATTACK_LBL_DIR  = os.path.join(ATTACK_ROOT, "labels/val")

# Class names (must match training order)
NAMES = [
    "plane","ship","storage tank","baseball diamond","tennis court",
    "basketball court","ground track field","harbor","bridge","large vehicle",
    "small vehicle","helicopter","roundabout","soccer ball field","swimming pool",
    "container crane"
]

# ========= ATTACK + EVAL SETTINGS =========
IMG_SIZE      = 1024
CONF_THRES    = 0.001
DEVICE        = 0  # 'cpu' if no GPU

# DE parameters: control attack strength vs. speed
MAX_ITER      = 25  # How many "generations" the algorithm runs
POP_SIZE      = 10  # How many candidate pixels in each generation

# ========= UTILITY FUNCTIONS =========
def ensure_clean_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)

def score_image(model, img_bgr):
    """
    Black-box 'badness' score (fitness function) we want to MINIMIZE.
    Returns the sum of confidences of all detections.
    """
    res = model.predict(source=img_bgr, imgsz=IMG_SIZE, conf=CONF_THRES,
                        device=DEVICE, half=True, verbose=False)[0]
    if res.boxes is None or len(res.boxes) == 0:
        return 0.0
    return float(res.boxes.conf.sum().item())

def de_one_pixel_attack(model, img_bgr):
    """
    Finds the best single-pixel attack using Differential Evolution.
    """
    h, w, _ = img_bgr.shape
    base_score = score_image(model, img_bgr)

    # 1. Define the function for DE to MINIMIZE
    def objective_function(pixel_params):
        # pixel_params is a 1D array: [x, y, r, g, b]
        x, y, r, g, b = [int(p) for p in pixel_params]

        # Ensure coordinates are within image bounds
        x = max(0, min(w - 1, x))
        y = max(0, min(h - 1, y))

        attacked_img = img_bgr.copy()
        attacked_img[y, x] = (b, g, r)  # Use BGR for cv2

        return score_image(model, attacked_img)

    # 2. Define the BOUNDS for each parameter (x, y, r, g, b)
    bounds = [
        (0, w - 1),  # x coordinate
        (0, h - 1),  # y coordinate
        (0, 255),    # Red channel
        (0, 255),    # Green channel
        (0, 255)     # Blue channel
    ]

    # 3. Run the Differential Evolution solver
    result = differential_evolution(
        func=objective_function,
        bounds=bounds,
        maxiter=MAX_ITER,
        popsize=POP_SIZE,
        disp=False
    )

    # 4. Extract the best attack and create the final image
    best_pixel = result.x.astype(int)
    final_attack_img = img_bgr.copy()
    x, y, r, g, b = best_pixel
    x = max(0, min(w - 1, x))
    y = max(0, min(h - 1, y))
    final_attack_img[y, x] = (b, g, r)

    attacked_score = result.fun  # The score of the best solution found

    return final_attack_img, attacked_score, base_score

# ========= SCRIPT EXECUTION =========
ensure_clean_dir(ATTACK_IMG_DIR)
ensure_clean_dir(ATTACK_LBL_DIR)

# Copy labels to the new attack directory
print("Copying labels...")
for p in glob.glob(os.path.join(VAL_LABELS_DIR, "*.txt")):
    shutil.copy2(p, ATTACK_LBL_DIR)

print("Loading model...")
model = YOLO(MODEL_PATH)

exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
img_paths = [p for p in glob.glob(os.path.join(VAL_IMAGES_DIR, "*")) if p.lower().endswith(exts)]
per_image_scores = []

print("Starting DE one-pixel attack...")
for ip in tqdm(img_paths, desc="DE One-pixel attacking"):
    img = cv2.imread(ip)
    if img is None:
        continue

    attacked, attacked_score, base_score = de_one_pixel_attack(model, img)

    base_name = os.path.basename(ip)
    cv2.imwrite(os.path.join(ATTACK_IMG_DIR, base_name), attacked)
    per_image_scores.append((base_name, base_score, attacked_score))

print(f"\n[OK] Saved attacked images to: {ATTACK_IMG_DIR}")

print("\nPausing for 15 seconds to allow file system to sync...")
time.sleep(15)

# Optional: Print the average score drop
drops = [b - a for _, b, a in per_image_scores if b > 0] # Avoid division by zero
if drops:
    print(f"Proxy score drop (mean) over {len(drops)} images: {np.mean(drops):.4f}")

# ========= FINAL EVALUATION =========
print("\nRunning evaluation on the attacked dataset...")
import yaml
DATA_YAML = "/content/de_attacked_tmp.yaml"
with open(DATA_YAML, "w") as f:
    yaml.safe_dump({
        "path": ATTACK_ROOT,
        "train": "images/val",
        "val": "images/val",
        "names": {i: n for i, n in enumerate(NAMES)},
        "nc": len(NAMES)
    }, f, sort_keys=False)

metrics_attack = model.val(
    data=DATA_YAML,
    split="val",
    imgsz=IMG_SIZE,
    conf=CONF_THRES,
    iou=0.5,
    device=DEVICE,
    half=True,
    verbose=False
)

print("\n== DE ATTACKED SET RESULTS ==")
print(f"mAP@0.50      : {metrics_attack.box.map50:.4f}")
print(f"mAP@0.50:0.95 : {metrics_attack.box.map:.4f}")
print(f"Precision     : {metrics_attack.box.mp:.4f}")
print(f"Recall        : {metrics_attack.box.mr:.4f}")

Copying labels...
Loading model...
Starting DE one-pixel attack...


DE One-pixel attacking:  78%|███████▊  | 293/374 [2:05:58<28:02, 20.77s/it]

WARNING ⚠️ NMS time limit 2.050s exceeded


DE One-pixel attacking: 100%|██████████| 374/374 [2:39:36<00:00, 25.61s/it]



[OK] Saved attacked images to: /content/drive/MyDrive/MN-20-Credit/dota-yolo/de_one_pixel_attack/images/val

Pausing for 15 seconds to allow file system to sync...
Proxy score drop (mean) over 374 images: 0.0739

Running evaluation on the attacked dataset...
Ultralytics 8.3.209 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 9.6±20.2 ms, read: 191.5±177.7 MB/s, size: 20069.3 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_one_pixel_attack/labels/val... 374 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 374/374 20.9it/s 17.9s
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_one_pixel_attack/images/val/P0003.png: 55 duplicate labels removed
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_one_pixel_attack/images/val/P0011.png: 92 duplicate labels removed
val: 

# 3 Pixel Attack

In [ ]:
import os
import shutil
import glob
import time
import cv2
import yaml
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from scipy.optimize import differential_evolution
import torch
import random

# ========= USER CONFIG (edit these paths if needed) =========
MODEL_PATH = "/content/drive/MyDrive/MN-20-Credit/models/yolov9/best.pt"
VAL_IMAGES_DIR = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val"
VAL_LABELS_DIR = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/labels/val"

ATTACK_ROOT = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/de_three_pixel_attack"
# inside ATTACK_ROOT we'll create:
# - baseline/images/val  (sampled originals)
# - baseline/preds       (baseline annotated images)
# - images/val           (attacked images)
# - labels/val           (copied labels for attacked set)
# - attacked_preds       (annotated attacked images)

# class names (map index->name)
NAMES = [
    "plane","ship","storage tank","baseball diamond","tennis court",
    "basketball court","ground track field","harbor","bridge","large vehicle",
    "small vehicle","helicopter","roundabout","soccer ball field","swimming pool",
    "container crane"
]

# ========= ATTACK + EVAL SETTINGS =========
IMG_SIZE = 1024
CONF_THRES = 0.001

# Device and half-precision
USE_CUDA = torch.cuda.is_available()
DEVICE = "cuda" if USE_CUDA else "cpu"
HALF = USE_CUDA
print(f"[INFO] Using device: {DEVICE}, half precision: {HALF}")

# DE parameters
MAX_ITER = 30
POP_SIZE = 20
DE_SEED = 42
DE_WORKERS = 1

# ========= REPRODUCIBILITY =========
random.seed(DE_SEED)
np.random.seed(DE_SEED)
torch.manual_seed(DE_SEED)
if USE_CUDA:
    torch.cuda.manual_seed_all(DE_SEED)
    torch.backends.cudnn.benchmark = True

# ========= UTILITY FUNCTIONS =========

def ensure_clean_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)


def score_image(model, img_bgr):
    try:
        res = model.predict(source=img_bgr, imgsz=IMG_SIZE, conf=CONF_THRES,
                            device=DEVICE, half=HALF, verbose=False)[0]
    except Exception:
        res = model.predict(source=img_bgr, imgsz=IMG_SIZE, conf=CONF_THRES,
                            device="cpu", half=False, verbose=False)[0]
    if getattr(res, "boxes", None) is None or len(res.boxes) == 0:
        return 0.0
    confs = getattr(res.boxes, "conf", None)
    if confs is None:
        return 0.0
    try:
        s = float(confs.sum().item())
    except Exception:
        s = float(np.array(confs).sum())
    return s


# DE three-pixel attack (returns attacked_img, attacked_score, base_score)
def de_three_pixel_attack(model, img_bgr, maxiter=MAX_ITER, popsize=POP_SIZE, seed=DE_SEED, workers=DE_WORKERS):
    h, w, _ = img_bgr.shape
    base_score = score_image(model, img_bgr)

    pixel_bounds = [(0, w - 1), (0, h - 1), (0, 255), (0, 255), (0, 255)]
    bounds = pixel_bounds * 3  # 3 pixels = 15 params

    # lightweight progress display
    from tqdm import tqdm as _tqdm
    pbar = _tqdm(total=maxiter, desc="DE generations", leave=False)

    def callback_progress(xk, convergence):
        try:
            pbar.update(1)
        except Exception:
            pass
        return False

    def objective_function(params):
        attacked_img = img_bgr.copy()
        for i in range(3):
            offset = i * 5
            x = int(np.round(params[offset + 0])); x = int(np.clip(x, 0, w - 1))
            y = int(np.round(params[offset + 1])); y = int(np.clip(y, 0, h - 1))
            r = int(np.clip(int(np.round(params[offset + 2])), 0, 255))
            g = int(np.clip(int(np.round(params[offset + 3])), 0, 255))
            b = int(np.clip(int(np.round(params[offset + 4])), 0, 255))
            attacked_img[y, x] = [b, g, r]
        return score_image(model, attacked_img)

    # try multiprocess workers, fallback to single
    result = None
    for attempt_workers in (workers, 1):
        try:
            result = differential_evolution(
                func=objective_function,
                bounds=bounds,
                maxiter=maxiter,
                popsize=popsize,
                seed=seed,
                disp=False,
                polish=True,
                workers=attempt_workers,
                callback=callback_progress
            )
            break
        except Exception as e:
            if attempt_workers == 1:
                pbar.close()
                raise
            else:
                print(f"[WARN] differential_evolution with workers={attempt_workers} failed: {e}. Retrying with workers=1...")
                pbar.close()
                time.sleep(1)

    pbar.close()

    best_params = result.x
    final_attack_img = img_bgr.copy()
    for i in range(3):
        offset = i * 5
        x = int(np.clip(int(np.round(best_params[offset + 0])), 0, w - 1))
        y = int(np.clip(int(np.round(best_params[offset + 1])), 0, h - 1))
        r = int(np.clip(int(np.round(best_params[offset + 2])), 0, 255))
        g = int(np.clip(int(np.round(best_params[offset + 3])), 0, 255))
        b = int(np.clip(int(np.round(best_params[offset + 4])), 0, 255))
        final_attack_img[y, x] = [b, g, r]

    attacked_score = float(result.fun)
    return final_attack_img, attacked_score, base_score


# ========= SCRIPT EXECUTION =========

# Prepare directories
ensure_clean_dir(ATTACK_ROOT)
BASELINE_ROOT = os.path.join(ATTACK_ROOT, "baseline")
ATTACK_IMG_DIR = os.path.join(ATTACK_ROOT, "images/val")
ATTACK_LBL_DIR = os.path.join(ATTACK_ROOT, "labels/val")
BASELINE_IMG_DIR = os.path.join(BASELINE_ROOT, "images/val")
BASELINE_LBL_DIR = os.path.join(BASELINE_ROOT, "labels/val")
BASELINE_PRED_DIR = os.path.join(BASELINE_ROOT, "preds")
ATTACKED_PRED_DIR = os.path.join(ATTACK_ROOT, "attacked_preds")

for d in (BASELINE_IMG_DIR, BASELINE_LBL_DIR, BASELINE_PRED_DIR, ATTACK_IMG_DIR, ATTACK_LBL_DIR, ATTACKED_PRED_DIR):
    os.makedirs(d, exist_ok=True)

print("Loading model...")
model = YOLO(MODEL_PATH)

# free some GPU memory (if using GPU)
if USE_CUDA:
    torch.cuda.empty_cache()

exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
all_img_paths = [p for p in glob.glob(os.path.join(VAL_IMAGES_DIR, "*")) if p.lower().endswith(exts)]

random.shuffle(all_img_paths)
num_to_test = max(1, int(len(all_img_paths) * 0.05))
sampled_img_paths = all_img_paths[:num_to_test]
print(f"[INFO] A random 5% sample will be tested: {len(sampled_img_paths)} images.\n")

# Copy corresponding labels and images into baseline and copy labels into ATTACK_LBL_DIR for attacked set
for img_p in sampled_img_paths:
    base = os.path.basename(img_p)
    lbl_name = os.path.splitext(base)[0] + ".txt"
    src_lbl = os.path.join(VAL_LABELS_DIR, lbl_name)
    # copy image -> baseline images
    shutil.copy2(img_p, os.path.join(BASELINE_IMG_DIR, base))
    # copy label -> baseline labels (if existing)
    if os.path.exists(src_lbl):
        shutil.copy2(src_lbl, os.path.join(BASELINE_LBL_DIR, lbl_name))
        shutil.copy2(src_lbl, os.path.join(ATTACK_LBL_DIR, lbl_name))

# ========= Baseline per-image predictions (annotated) =========
print("Running baseline per-image predictions and saving annotated images...")
for img_p in tqdm(sampled_img_paths, desc="Baseline preds"):
    img = cv2.imread(img_p)
    if img is None:
        continue
    # predict once and save annotated image
    try:
        res = model.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRES, device=DEVICE, half=HALF, verbose=False)[0]
    except Exception:
        res = model.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRES, device="cpu", half=False, verbose=False)[0]
    ann = res.plot()  # annotated image as BGR
    cv2.imwrite(os.path.join(BASELINE_PRED_DIR, os.path.basename(img_p)), ann)

# Baseline evaluation using ultralytics val() on the sampled set
DATA_YAML_BASE = os.path.join(ATTACK_ROOT, "de_baseline_tmp.yaml")
with open(DATA_YAML_BASE, "w") as f:
    yaml.safe_dump({
        "path": BASELINE_ROOT,
        "train": "images/val",
        "val": "images/val",
        "names": {i: n for i, n in enumerate(NAMES)},
        "nc": len(NAMES)
    }, f, sort_keys=False)

print("\nEvaluating baseline (sampled 5%) with model.val()...")
metrics_base = model.val(data=DATA_YAML_BASE, split="val", imgsz=IMG_SIZE, conf=CONF_THRES,
                         iou=0.5, device=DEVICE, half=HALF, verbose=False)

base_map50 = float(metrics_base.box.map50)
base_map = float(metrics_base.box.map)
base_prec = float(metrics_base.box.mp)
base_rec = float(metrics_base.box.mr)

print("\n== BASELINE (SAMPLED 5%) RESULTS ==")
print(f"mAP@0.50 : {base_map50:.4f}")
print(f"mAP@0.50:0.95 : {base_map:.4f}")
print(f"Precision : {base_prec:.4f}")
print(f"Recall : {base_rec:.4f}")

# ========= RUN ATTACK ON SAMPLED IMAGES =========
print("\nStarting DE three-pixel attack on sampled images...")
per_image_scores = []
for ip in tqdm(sampled_img_paths, desc="Attacking images"):
    img = cv2.imread(ip)
    if img is None:
        continue

    attacked, attacked_score, base_score = de_three_pixel_attack(
        model, img, maxiter=MAX_ITER, popsize=POP_SIZE, seed=DE_SEED, workers=DE_WORKERS
    )

    base_name = os.path.basename(ip)
    # save attacked image
    cv2.imwrite(os.path.join(ATTACK_IMG_DIR, base_name), attacked)
    per_image_scores.append((base_name, base_score, attacked_score))

print(f"\n[OK] Saved attacked images to: {ATTACK_IMG_DIR}")
print("\nPausing briefly to allow file system to sync...")
time.sleep(5)

# compute proxy drops
drops = [base - attacked for _, base, attacked in per_image_scores if base > 0]
if drops:
    print(f"Proxy score drop (mean) over {len(drops)} images: {np.mean(drops):.4f}")

# ========= Attacked predictions (annotated) =========
print("\nRunning predictions on attacked images and saving annotated images...")
attacked_img_paths = [os.path.join(ATTACK_IMG_DIR, os.path.basename(x)) for x in sampled_img_paths]
for aip in tqdm(attacked_img_paths, desc="Attacked preds"):
    img = cv2.imread(aip)
    if img is None:
        continue
    try:
        res = model.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRES, device=DEVICE, half=HALF, verbose=False)[0]
    except Exception:
        res = model.predict(source=img, imgsz=IMG_SIZE, conf=CONF_THRES, device="cpu", half=False, verbose=False)[0]
    ann = res.plot()
    cv2.imwrite(os.path.join(ATTACKED_PRED_DIR, os.path.basename(aip)), ann)

# ========= EVALUATION ON ATTACKED SET =========
DATA_YAML_ATTACK = os.path.join(ATTACK_ROOT, "de_attacked_tmp.yaml")
with open(DATA_YAML_ATTACK, "w") as f:
    yaml.safe_dump({
        "path": ATTACK_ROOT,
        "train": "images/val",
        "val": "images/val",
        "names": {i: n for i, n in enumerate(NAMES)},
        "nc": len(NAMES)
    }, f, sort_keys=False)

print("\nEvaluating attacked set with model.val()...")
metrics_attack = model.val(data=DATA_YAML_ATTACK, split="val", imgsz=IMG_SIZE, conf=CONF_THRES,
                           iou=0.5, device=DEVICE, half=HALF, verbose=False)

atk_map50 = float(metrics_attack.box.map50)
atk_map = float(metrics_attack.box.map)
atk_prec = float(metrics_attack.box.mp)
atk_rec = float(metrics_attack.box.mr)

print("\n== DE ATTACKED SET RESULTS ==")
print(f"mAP@0.50 : {atk_map50:.4f}")
print(f"mAP@0.50:0.95 : {atk_map:.4f}")
print(f"Precision : {atk_prec:.4f}")
print(f"Recall : {atk_rec:.4f}")

# ========= COMPARISON SUMMARY =========
print("\n== SUMMARY (Baseline vs Attacked on sampled 5%) ==")
print(f"mAP@0.50 : baseline={base_map50:.4f}  |  attacked={atk_map50:.4f}  |  drop={base_map50 - atk_map50:.4f}")
print(f"mAP@0.50:0.95 : baseline={base_map:.4f}  |  attacked={atk_map:.4f}  |  drop={base_map - atk_map:.4f}")
print(f"Precision : baseline={base_prec:.4f}  |  attacked={atk_prec:.4f}  |  drop={base_prec - atk_prec:.4f}")
print(f"Recall : baseline={base_rec:.4f}  |  attacked={atk_rec:.4f}  |  drop={base_rec - atk_rec:.4f}")

# Save per-image proxy drops CSV
csv_out = os.path.join(ATTACK_ROOT, "per_image_proxy_scores.csv")
with open(csv_out, "w") as f:
    f.write("image,base_score,attacked_score,drop\n")
    for base_name, base_score, attacked_score in per_image_scores:
        f.write(f"{base_name},{base_score:.6f},{attacked_score:.6f},{(base_score-attacked_score):.6f}\n")

print(f"\nPer-image proxy scores written to: {csv_out}")
print(f"Annotated baseline preds: {BASELINE_PRED_DIR}")
print(f"Annotated attacked preds: {ATTACKED_PRED_DIR}")

print("\nDone.")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
[INFO] Using device: cuda, half precision: True
Loading model...
[INFO] A random 5% sample will be tested: 18 images.

Running baseline per-image predictions and saving annotated images...


Baseline preds: 100%|██████████| 18/18 [00:28<00:00,  1.59s/it]



Evaluating baseline (sampled 5%) with model.val()...
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.9±0.4 ms, read: 481.5±159.6 MB/s, size: 6726.2 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/baseline/labels/val... 18 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 18/18 42.6it/s 0.4s
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/baseline/images/val/P0042.png: 60 duplicate labels removed
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/baseline/images/val/P0217.png: 39 duplicate labels removed
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/baseline/images/val/P0233.png: 84 duplicate labels removed
va

Attacking images: 100%|██████████| 18/18 [13:32<00:00, 45.11s/it]



[OK] Saved attacked images to: /content/drive/MyDrive/MN-20-Credit/dota-yolo/de_three_pixel_attack/images/val

Pausing briefly to allow file system to sync...
Proxy score drop (mean) over 18 images: 0.1613

Running predictions on attacked images and saving annotated images...


Attacked preds: 100%|██████████| 18/18 [00:16<00:00,  1.08it/s]


Evaluating attacked set with model.val()...
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 589.5±187.6 MB/s, size: 10553.1 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/labels/val... 18 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 18/18 34.9it/s 0.5s
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/images/val/P0042.png: 60 duplicate labels removed
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/images/val/P0217.png: 39 duplicate labels removed
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/images/val/P0233.png: 84 duplicate labels removed
val: /content/drive/.shortcut-targets-by-id/1zpTDLsM5hxDp5YzIh-V3E0q2h9S7058N/MN-20-Credit/dota-yolo/de_three_pixel_attack/images/val/P0294.png: 13 duplicate labels removed
v